In [ ]:
# Code to wrap output
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [ ]:
# Import libraries
import pandas as pd
import numpy as np

In [ ]:
# Install ktrain
!pip install ktrain

     |████████████████████████████████| 25.3MB 72.9MB/s 
     |████████████████████████████████| 6.8MB 51.0MB/s 
     |████████████████████████████████| 983kB 53.2MB/s 
     |████████████████████████████████| 266kB 41.9MB/s 
     |████████████████████████████████| 1.9MB 48.2MB/s 
     |████████████████████████████████| 1.2MB 44.2MB/s 
     |████████████████████████████████| 471kB 49.2MB/s 
     |████████████████████████████████| 3.3MB 47.9MB/s 
     |████████████████████████████████| 901kB 37.6MB/s 
  Created wheel for ktrain: filename=ktrain-0.26.2-cp37-none-any.whl size=25277794 sha256=8550be509b38440f05e3196fa6b36d86e39625ce9d777233446ff6ce895704e6
  Stored in directory: /root/.cache/pip/wheels/67/2e/f1/c72afa08df8b2d984b910dea228902ce81dae4511afe9fafd2
  Created wheel for langdetect: filename=langdetect-1.0.8-cp37-none-any.whl size=993193 sha256=23b2ec0557b7ca9a8edc8757a95ca3cd27f43047c9132907d2cae742d392aa36
  Stored in directory: /root/.cache/pip/wheels/8d/b3/aa/6d99de9f3841d7d3d

In [ ]:
# Mount Google Drive to Colab
from google.colab import drive
drive.mount('/content/drive')

# # Change to correct directory
import os
os.chdir("/content/drive/MyDrive")
!ls

Mounted at /content/drive
 bert
'Colab Notebooks'
 covid_tweets_cleaned.csv
"Dawson's Documents.pdf"
'Part 2: Predicting and Analyzing Covid-19 Tweets V1.ipynb'
'Pre-wedding Shoot'
'Quadratic Graphs Booklet.docx'
'Quadratic Graphs Booklet.docx.gdoc'
'Resume (1).gdoc'
 Resume.gdoc
 sentiment140_test.csv
 sentiment140_train.csv
 Takeout
 uscities.csv
 us_state_capitals.csv
 us_state_vaccinations.csv
 vaccination_all_tweets.csv
'Video for Bell.mp4'


In [ ]:
# Import files
vac_tweets = pd.read_csv('vaccination_all_tweets.csv')
us_vac = pd.read_csv('us_state_vaccinations.csv')
# us_cities = pd.read_csv('uscities.csv') # Extra dataset from https://simplemaps.com/data/us-cities 
us_states = pd.read_csv('us_state_capitals.csv') 

In [ ]:
# Display vaccination tweets
vac_tweets.head()

,id,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,retweets,favorites,is_retweet
0,1340539111971516416,Rachel Roh,"La Crescenta-Montrose, CA",Aggregator of Asian American news; scanning di...,2009-04-08 17:52:46,405,1692,3247,False,2020-12-20 06:06:44,Same folks said daikon paste could treat a cyt...,['PfizerBioNTech'],Twitter for Android,0,0,False
1,1338158543359250433,Albert Fong,"San Francisco, CA","Marketing dude, tech geek, heavy metal & '80s ...",2009-09-21 15:27:30,834,666,178,False,2020-12-13 16:27:13,While the world has been on the wrong side of ...,NaN,Twitter Web App,1,1,False
2,1337858199140118533,eli🇱🇹🇪🇺👌,Your Bed,"heil, hydra 🖐☺",2020-06-25 23:30:28,10,88,155,False,2020-12-12 20:33:45,#coronavirus #SputnikV #AstraZeneca #PfizerBio...,"['coronavirus', 'SputnikV', 'AstraZeneca', 'Pf...",Twitter for Android,0,0,False
3,1337855739918835717,Charles Adler,"Vancouver, BC - Canada","Hosting ""CharlesAdlerTonight"" Global News Radi...",2008-09-10 11:28:53,49165,3933,21853,True,2020-12-12 20:23:59,"Facts are immutable, Senator, even when you're...",NaN,Twitter Web App,446,2129,False
4,1337854064604966912,Citizen News Channel,NaN,Citizen News Channel bringing you an alternati...,2020-04-23 17:58:42,152,580,1473,False,2020-12-12 20:17:19,Explain to me again why we need a vaccine @Bor...,"['whereareallthesickpeople', 'PfizerBioNTech']",Twitter for iPhone,0,0,False


In [ ]:
# Set no. of rows displayed
pd.set_option("display.max_rows", 10)

# Display relevant columns (user_location, date, text/tweet)
vac_tweets = vac_tweets[["user_location","date","text"]]

# Remove rows with user_location as NaN
vac_tweets = vac_tweets.dropna().copy()

# Convert strings in user_location to lowercase
vac_tweets["user_location"] = vac_tweets["user_location"].str.lower().copy()

# Convert date to year and month
vac_tweets["date"] = pd.to_datetime(vac_tweets["date"]).dt.to_period('M')

vac_tweets

,user_location,date,text
0,"la crescenta-montrose, ca",2020-12,Same folks said daikon paste could treat a cyt...
1,"san francisco, ca",2020-12,While the world has been on the wrong side of ...
2,your bed,2020-12,#coronavirus #SputnikV #AstraZeneca #PfizerBio...
3,"vancouver, bc - canada",2020-12,"Facts are immutable, Senator, even when you're..."
5,"birmingham, england",2020-12,Does anyone have any useful advice/guidance fo...
...,...,...,...
40165,italia,2021-03,i want #SputnikV
40166,"dublin, ireland",2021-03,"Selling: #NitrileGloves, #1860 #FaceMasks, #Va..."
40167,"geneva, switzerland",2021-03,Limited vaccine manufacturing capacity is a ma...
40169,sri lanka,2021-03,"State Minister of Production, Supply and Regul..."


In [ ]:
# # Select relevant columns
# us_cities = us_cities[["city","state_id","state_name"]]

# # Convert to lowercase
# us_cities ["city"] = us_cities ["city"].str.lower()
# us_cities ["state_name"] = us_cities ["state_name"].str.lower()

# #### Don't include state abbr / id as will get a lot of unwanted matches

# us_cities

# # Obtain US states as a list
# states = list(pd.unique(us_cities["state_name"]))
# cities = dict()

# # for s in states:
# #   c = list(us_cities[us_cities["state_name"] == s]["city"])
# #   c.append(s)
# #   c = "\\b[,]* | \\b".join(c)
# #   c = "\\b" + c + "\\b[,]*"
# #   # print(c)
# #   cities[s] = c
# #   # break

# cities["new york"]

In [ ]:
# Display all rows
pd.set_option("display.max_rows", 10)

# Combine state, id and capital into regex pattern
us_states["combined"] = "\\b" + us_states["state"] + "\\b|\\b" + us_states["id"] + "\\b|\\b" + us_states["capital"] +"\\b"

# Convert to lowercase
us_states["combined"] = us_states["combined"].str.lower()

# Display
us_states

,state,id,capital,combined
0,Alabama,AL,Montgomery,\balabama\b|\bal\b|\bmontgomery\b
1,Alaska,AK,Juneau,\balaska\b|\bak\b|\bjuneau\b
2,Arizona,AZ,Phoenix,\barizona\b|\baz\b|\bphoenix\b
3,Arkansas,AR,Little Rock,\barkansas\b|\bar\b|\blittle rock\b
4,California,CA,Sacramento,\bcalifornia\b|\bca\b|\bsacramento\b
...,...,...,...,...
45,Virginia,VA,Richmond,\bvirginia\b|\bva\b|\brichmond\b
46,Washington,WA,Olympia,\bwashington\b|\bwa\b|\bolympia\b
47,West Virginia,WV,Charleston,\bwest virginia\b|\bwv\b|\bcharleston\b
48,Wisconsin,WI,Madison,\bwisconsin\b|\bwi\b|\bmadison\b


In [ ]:
us_states["combined"][24]

'\\bmissouri\\b|\\bmo\\b|\\bjefferson city\\b'

In [ ]:
# for i in range(len(us_states)):
#   pd.set_option("display.max_rows", 20, "display.max_columns", 20)
#   print(us_states["state"][i])
#   display(vac_tweets[vac_tweets["user_location"].str.contains(us_states["combined"][i], regex = True)])
#   print()  

In [ ]:
df_by_state = dict()

for _, row in us_states.iterrows():
  df_by_state[row["state"]] = vac_tweets[vac_tweets["user_location"].str.contains(row["combined"], regex = True)]
  df_by_state[row["state"]]["state"] = row["state"]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
# Further filtering Alabama
# pd.set_option("display.max_rows", 10)
df_by_state["Alabama"] = df_by_state["Alabama"][df_by_state["Alabama"]["user_location"].str.contains("\\b, al\\b|alabama|montgomery", regex = True)]

In [ ]:
# Further filtering Delaware
# pd.set_option("display.max_rows", None)
df_by_state["Delaware"] = df_by_state["Delaware"][df_by_state["Delaware"]["user_location"].str.contains("\\b, de\\b|delaware|dover", regex = True)]

In [ ]:
# Further filtering Indiana
df_by_state["Indiana"] = df_by_state["Indiana"][df_by_state["Indiana"]["user_location"].str.contains("\\bindiana\\b|indianapolis", regex = True)]

In [ ]:
# Further filtering Louisiana
df_by_state["Louisiana"] = df_by_state["Louisiana"][df_by_state["Louisiana"]["user_location"].str.contains("\\b, la\\b$|\\blouisiana\\b|baton rouge", regex = True)]

In [ ]:
# Further filtering Maine
df_by_state["Maine"] = df_by_state["Maine"][df_by_state["Maine"]["user_location"].str.contains("\\b, me\\b$|\\bmaine\\b", regex = True)]

In [ ]:
# Further filtering Montana
df_by_state["Montana"] = df_by_state["Montana"][df_by_state["Montana"]["user_location"].str.contains("\\bmt\\b$|\\bmontana\\b|helena", regex = True)]

In [ ]:
# Further filtering Nebraska
df_by_state["Nebraska"] = df_by_state["Nebraska"][df_by_state["Nebraska"]["user_location"].str.contains("\\bne\\b|\\bnebraska\\b|lincoln$", regex = True)]

In [ ]:
# Further filtering South Carolina
df_by_state["South Carolina"] = df_by_state["South Carolina"][df_by_state["South Carolina"]["user_location"].str.contains("\\bsc\\b|\\bsouth carolina\\b$", regex = True)]
# df_by_state["South Carolina"]

In [ ]:
# Print the number of tweets for each state

# for state, df in df_by_state.items():
#   print(state)
#   print(df.shape[0])
#   print()

In [ ]:
df_cleaned = pd.DataFrame()

for state, df in df_by_state.items():
  df_cleaned = pd.concat([df_cleaned, df[["date","text","state"]]])
  
df_cleaned

,date,text,state
511,2020-12,Got the COVID vaccine today. This is amazing. ...,Alabama
1372,2020-12,“We are protecting the high risk.” -@realDonal...,Alabama
1736,2020-12,I'm grateful for all of the healthcare workers...,Alabama
1829,2020-12,#fomonomo #COVAX #Pfizer #PfizerBioNTech #Pfiz...,Alabama
3061,2021-01,"Dr Fun's I Feel Good - My effort at a PSA, ple...",Alabama
...,...,...,...
27209,2021-03,I was blessed to got my first Fauci Ouchie ge...,Wyoming
29367,2021-03,yes can we put this Virus behind us #OxfordAst...,Wyoming
32022,2021-03,#secondshot tomorrow my mom and I will receive...,Wyoming
33269,2021-03,"Just FYI, y’all can feel bitter towards me now...",Wyoming


In [ ]:
### Clean dataset ###

## Step 1: Remove Twitter handles
df_cleaned["cleaned_tweet"] = df_cleaned["text"].str.replace('@[\w]*',"")

# Remove leading and trailing whitespaces
df_cleaned["cleaned_tweet"] = df_cleaned["cleaned_tweet"].str.strip() 

## Step 2: Remove URLS
pattern = 'https?://[A-Za-z0-9./]+'
df_cleaned["cleaned_tweet"] = df_cleaned["cleaned_tweet"].str.replace(pattern,"")

# Remove leading and trailing whitespaces
df_cleaned["cleaned_tweet"] = df_cleaned["cleaned_tweet"].str.strip() 

## Step 3: Remove punctuations, numbers and special characters (Remove # but keeping words after #)
pattern = '[^a-zA-Z\s]'
df_cleaned["cleaned_tweet"] = df_cleaned["cleaned_tweet"].str.replace(pattern,"")

# Remove leading and trailing whitespaces
df_cleaned["cleaned_tweet"] = df_cleaned["cleaned_tweet"].str.strip() 

# Remove extra spaces
df_cleaned["cleaned_tweet"] = df_cleaned["cleaned_tweet"].str.replace("\s{2,10}"," ")

## Step 4: Convert all words to lowercase
df_cleaned["cleaned_tweet"] = df_cleaned["cleaned_tweet"].str.lower()

df_cleaned

,date,text,state,cleaned_tweet
511,2020-12,Got the COVID vaccine today. This is amazing. ...,Alabama,got the covid vaccine today this is amazing so...
1372,2020-12,“We are protecting the high risk.” -@realDonal...,Alabama,we are protecting the high risk bull shit if t...
1736,2020-12,I'm grateful for all of the healthcare workers...,Alabama,im grateful for all of the healthcare workers ...
1829,2020-12,#fomonomo #COVAX #Pfizer #PfizerBioNTech #Pfiz...,Alabama,fomonomo covax pfizer pfizerbiontech pfizercov...
3061,2021-01,"Dr Fun's I Feel Good - My effort at a PSA, ple...",Alabama,dr funs i feel good my effort at a psa please ...
...,...,...,...,...
27209,2021-03,I was blessed to got my first Fauci Ouchie ge...,Wyoming,i was blessed to got my first fauci ouchie get...
29367,2021-03,yes can we put this Virus behind us #OxfordAst...,Wyoming,yes can we put this virus behind us oxfordastr...
32022,2021-03,#secondshot tomorrow my mom and I will receive...,Wyoming,secondshot tomorrow my mom and i will receive ...
33269,2021-03,"Just FYI, y’all can feel bitter towards me now...",Wyoming,just fyi yall can feel bitter towards me nowiv...


In [ ]:
############# 

# Import ktrain
import ktrain

# Load your predictor
predictor = ktrain.load_predictor('/content/drive/MyDrive/bert/predictor-autofit')

In [ ]:
############# 

# Predict sentiment of tweet 
prediction = predictor.predict(df_cleaned["cleaned_tweet"].tolist())

In [ ]:
# Combine prediction with dataframe
df_cleaned.reset_index(inplace = True, drop = True)
df_cleaned["prediction"] = pd.Series(prediction)
df_cleaned = df_cleaned[["date","state","cleaned_tweet","prediction"]]
df_cleaned

,date,state,cleaned_tweet,prediction
0,2020-12,Alabama,got the covid vaccine today this is amazing so...,positive
1,2020-12,Alabama,we are protecting the high risk bull shit if t...,positive
2,2020-12,Alabama,im grateful for all of the healthcare workers ...,positive
3,2020-12,Alabama,fomonomo covax pfizer pfizerbiontech pfizercov...,negative
4,2021-01,Alabama,dr funs i feel good my effort at a psa please ...,positive
...,...,...,...,...
6899,2021-03,Wyoming,i was blessed to got my first fauci ouchie get...,positive
6900,2021-03,Wyoming,yes can we put this virus behind us oxfordastr...,negative
6901,2021-03,Wyoming,secondshot tomorrow my mom and i will receive ...,positive
6902,2021-03,Wyoming,just fyi yall can feel bitter towards me nowiv...,positive


In [ ]:
# Save dataframe to a csv file
df_cleaned.to_csv('covid_tweets_cleaned.csv', index=False)

In [ ]:
!pip install git+https://github.com/amaiya/eli5@tfkeras_0_10_1

  Cloning https://github.com/amaiya/eli5 (to revision tfkeras_0_10_1) to /tmp/pip-req-build-4q6tfyif
  Running command git clone -q https://github.com/amaiya/eli5 /tmp/pip-req-build-4q6tfyif
  Running command git checkout -b tfkeras_0_10_1 --track origin/tfkeras_0_10_1
  Switched to a new branch 'tfkeras_0_10_1'
  Branch 'tfkeras_0_10_1' set up to track remote branch 'tfkeras_0_10_1' from 'origin'.
  Created wheel for eli5: filename=eli5-0.10.1-py2.py3-none-any.whl size=106832 sha256=12d307bbaee47ab4843a465726a1b35646cd6df96c043439535dced5783aee09
  Stored in directory: /tmp/pip-ephem-wheel-cache-o4dujmec/wheels/51/59/0a/0f48442b8d209583a4453580938d7ba2270aca40edacee6d45
Successfully built eli5


In [ ]:
# See explaining text classification
predictor.explain(df_cleaned["cleaned_tweet"].tolist()[2])

In [ ]:
# # Predict with test data
# predicted_labels = predictor.predict(df_test["cleaned_tweet"].tolist())
# accuracy = np.sum(df_test["sentiment"] == predicted_labels)/len(predicted_labels)
# print("Test accuracy: " + str(round(accuracy*100)) + "%")

In [1]:
# Display US Vaccinations Record
# us_vac.head()